<a href="https://colab.research.google.com/github/ayusinelnik/narratives-at-conflict/blob/main/07_MFC_Generate_Lexicons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Set Up

In [ ]:
import csv
import pandas as pd
import numpy as np
import re
from collections import Counter, defaultdict
from nltk import tokenize
import nltk
nltk.download('punkt')

from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive/


Combine all issues into one file

In [ ]:
import os

folder_path = '/content/drive/MyDrive/Research_Thesis_Bocconi_2023/mfc_v4.0/'


files = [f for f in os.listdir(folder_path) if f.endswith('_labeled.json')]

merged_data = pd.DataFrame()

for file in files:
    file_path = os.path.join(folder_path, file)
    current_data = pd.read_json(file_path)
    print(current_data.info())

    merged_data = pd.concat([merged_data, current_data], axis=1)

merged_data.info()

In [ ]:
# tokenize and remove stopword from the MFC texts

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')


def process_text(text):
    tokenized_text = tokenize.word_tokenize(text)
    return [t for t in tokenized_text if not t.replace(",", "").replace(".","").isdigit() \
                and not t in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# produce counts over frames, articles

def do_counts(data):
    corpus_counter = Counter()
    code_to_counter = defaultdict(Counter)
    article_counter = Counter()
    article_count = 0

    for column in data.columns:
        assert "framing" in data[column]["annotations"]

        text = data[column]["text"].lower()
        article_counter.update(set(tokenize.word_tokenize(text)))
        article_count += 1

        for annotation_set in data[column]["annotations"]["framing"]:
            corpus_counter.update(process_text(text))

            for frame in data[column]["annotations"]["framing"][annotation_set]:
                coded_text = text[int(frame["start"]):int(frame["end"])]
                code_to_counter[frame["code"]].update(process_text(coded_text))

    return corpus_counter, code_to_counter, article_counter, article_count
    # above: words counter over all frames, words counter by frame, all words in all article texts, number of articles

In [ ]:

def words_to_pmi(background_counter, corpus_counter, code_to_counter, to_return_count = 250):
    frame_count = sum([code_to_counter[k] for k in code_to_counter])
    background_counter = sum(corpus_counter.values())

    word_to_pmi = {}
    for word in code_to_counter:
        # means it is a partial word or is infrequent
        if not word in corpus_counter:
            continue

        # number of times word appears with this frame
        # divide by number of words in frame = p( y | x)
        p_y_x = code_to_counter[word] / float(frame_count) # frequency of this words in the frame divided by number of words in the frame

        # number of times word appears at all / number of words in corpus = p(y)
        p_y = corpus_counter[word] / float(background_counter)

        assert (p_y_x > 0 and p_y_x < 1), str(p_y_x) + " " +  word
        assert (p_y > 0 and p_y < 1), str(p_y) + " " +  word

        word_to_pmi[word] = math.log(p_y_x / p_y)

    return sorted(word_to_pmi, key=word_to_pmi.get, reverse=True)[:to_return_count]

In [ ]:
# execution time approx 10 min

corpus_counter, code_to_counter, article_counter, article_count  = do_counts(merged_data)
# words counter over all frames, words counter by frame, all words and in how many articles they are, number of articles


We discard all words that occur in fewer than 0.5% of documents or in more than 98% of documents.




In [ ]:
# find words to cut (too frequent, too infrenquent) -- this should happen before PMI

def get_words_to_cut(article_count, article_counter, min_cutoff=1000, top_cutoff=50): #default cutoffs, will be overriden later

    min_num_articles = int(article_count / min_cutoff)
    max_num_articles = article_count - int(article_count / top_cutoff)

    words_to_cut = [w for w in article_counter if article_counter[w] < min_num_articles or
                    article_counter[w] > max_num_articles]
    return words_to_cut


In [ ]:
# cut infrequent and frequent words -- update the corpus_counter, that stores all frames words

cut_words = get_words_to_cut(article_count, article_counter, 200, 50) # override the defaults

corpus_counter = Counter({c:corpus_counter[c] for c in corpus_counter if not c in cut_words})
# execution time approx 10 min

In [ ]:
# calculate PMI
import math
code_to_lex = {}
background_counter = sum(corpus_counter.values())
for c in code_to_counter:

  if str(c).endswith(('.1', '.2')): #those are the frame codes for main frame and headline frame that we disregard
    continue
  code_to_lex[c] = words_to_pmi(background_counter, corpus_counter, code_to_counter[c], to_return_count = 250)


In [ ]:
#saving F_base

import json

file_path = '/content/drive/MyDrive/Research_Thesis_Bocconi_2023/code_to_lex.json'

with open(file_path, 'w') as json_file:
    json.dump(code_to_lex, json_file)

# Translate F_base to Russian

In [ ]:
!pip install python-dotenv

In [ ]:
from google.cloud import translate_v2 as translate
import os
from dotenv import load_dotenv

load_dotenv()

def translate_text_google_api(text, target_language='ru'):
    try:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""
        client = translate.Client()

        result = client.translate(text, target_language=target_language)
        translation = result['translatedText']
        return text, translation
    except Exception as e:
        print(f"Translation error for '{text}': {e}")
        return text, None

def translate_structure_google_api(structure):
    translated_structure = {}
    total_words_translated = 0

    for key, word_list in structure.items():
        translated_words = []
        for word in word_list:
            original, translation = translate_text_google_api(word)
            translated_words.append(translation)
            if translation is not None:
                total_words_translated += 1
                #print(f"Word '{word}' translated to '{translation}' successfully. Total words translated: {total_words_translated}")

        translated_structure[key] = translated_words

    return translated_structure

translated_structure = translate_structure_google_api(code_to_lex)



In [ ]:
import json

file_path = '/content/drive/MyDrive/Research_Thesis_Bocconi_2023/field_2018/code_to_lex_trans_ru.json'

with open(file_path, 'w') as json_file:
    json.dump(translated_structure, json_file)

# Train embeddings

In [ ]:
# change the training corpus to a relevant one when ready

import pandas as pd
import bz2

file_path = '/content/drive/MyDrive/Research_Thesis_Bocconi_2023/lenta-ru-news.csv.bz2'

with bz2.open(file_path, 'rt', encoding='utf-8') as f: # decompresses the file
    ru_background_corpus = pd.read_csv(f)


<ipython-input-37-789314f19d15>:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ru_background_corpus = pd.read_csv(f)


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')

texts = ru_background_corpus['text'].astype(str).tolist()


In [ ]:
# sample as needed
texts = texts[:10000] # execution for 100k time approx. 7 min

tokenized_texts = [word_tokenize(text.lower()) for text in texts]

stop_words = set(stopwords.words('russian'))

tokenized_texts = [
    [word for word in text if word.isalnum() and word not in stop_words]
    for text in tokenized_texts
]

model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=200,
    window=5,
    min_count=1,
    workers=5
)

model.save("/content/drive/MyDrive/Research_Thesis_Bocconi_2023/word2vec_model_ru.model")


In [ ]:

def get_article_top_words(input_texts):
    stop_words = set(stopwords.words('russian'))
    c = Counter()
    article_counter = Counter()
    num_articles = 0

    for text in input_texts:
        words = word_tokenize(text)

        words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

        c.update(words)
        article_counter.update(set(words))
        num_articles += 1

    return c, num_articles, article_counter

In [ ]:
#c, num_articles, article_counter = get_article_top_words(texts)

In [ ]:

top_words, num_articles, article_counter = get_article_top_words(texts)
#how many times a word appeared in all texts, # article total, in how many articles has a word appeared

vocab = sorted(top_words, key=top_words.get, reverse = True)[:50000]

In [ ]:
##### use this to only filter the F_base_translated to the limits of the background corpus vocab

from gensim.models import KeyedVectors

def seeds_to_real_lex(raw_lex, model, vocab):
    wv_model = Word2Vec.load(model)

    # Iterate over words in raw_lex
    filtered_seeds = [word for word in raw_lex if word in vocab and word in wv_model.wv]

    return filtered_seeds

In [ ]:
model = "/content/drive/MyDrive/Research_Thesis_Bocconi_2023/word2vec_model_ru.model"

filtered_code_to_lex = {key: seeds_to_real_lex(value, model, vocab) for key, value in code_to_lex.items()}

In [ ]:
code_to_lex

In [ ]:
def cluster_seeds(wv, seeds, topn, threshold, num_clusters=1):
    X = [wv[s] for s in seeds]
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X)
    expanded_seeds = []
    for center in kmeans.cluster_centers_:
        expanded_seeds += [x[0] for x in wv.most_similar(positive=[center], topn=topn) if x[1] >= threshold]
        #expanded_seeds += [x for x in seeds if (1 - cosine(center, wv[x])) >= threshold]
    return set(expanded_seeds)

In [ ]:
# final execution

from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine

model_path = "/content/drive/MyDrive/Research_Thesis_Bocconi_2023/word2vec_model_ru.model"
wv_model = Word2Vec.load(model_path)

expanded_seeds_dict = {}

for key in filtered_code_to_lex:
    expanded_seeds_dict[key] = cluster_seeds(wv_model.wv, filtered_code_to_lex[key], topn=1000, threshold=0.7, num_clusters=1)

# Print the number of elements for each key
for key, expanded_seeds in expanded_seeds_dict.items():
    print(f"{key}: {len(expanded_seeds)} elements")

# Query expansion

In [ ]:
# Again remove the words that appear in 98% and 0,5% of articles

cut_words = get_words_to_cut(num_articles, article_counter, 200, 50) # values are adequate for a sample of 10k texts

for key, values in expanded_seeds_dict.items():
    expanded_seeds_dict[key] = [value for value in values if value not in cut_words]

In [ ]:
#saving F_base_translated_expanded

import json

file_path = '/content/drive/MyDrive/Research_Thesis_Bocconi_2023/expanded_seeds_dict.json'

with open(file_path, 'w') as json_file:
    json.dump(expanded_seeds_dict, json_file)